<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Текущий-уровень-потребительской-лояльности-(Net-Promoter-Score)" data-toc-modified-id="Текущий-уровень-потребительской-лояльности-(Net-Promoter-Score)-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Текущий уровень потребительской лояльности (Net Promoter Score)</a></span><ul class="toc-item"><li><span><a href="#Описание-проекта" data-toc-modified-id="Описание-проекта-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Описание проекта</a></span></li><li><span><a href="#Установка-библиотек-и-загрузка-данных." data-toc-modified-id="Установка-библиотек-и-загрузка-данных.-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Установка библиотек и загрузка данных.</a></span><ul class="toc-item"><li><span><a href="#Установка-библиотек." data-toc-modified-id="Установка-библиотек.-1.2.1"><span class="toc-item-num">1.2.1&nbsp;&nbsp;</span>Установка библиотек.</a></span></li><li><span><a href="#Загрузка-данных" data-toc-modified-id="Загрузка-данных-1.2.2"><span class="toc-item-num">1.2.2&nbsp;&nbsp;</span>Загрузка данных</a></span></li><li><span><a href="#Формирование-SQL-запроса." data-toc-modified-id="Формирование-SQL-запроса.-1.2.3"><span class="toc-item-num">1.2.3&nbsp;&nbsp;</span>Формирование SQL запроса.</a></span></li><li><span><a href="#Создание-таблицы." data-toc-modified-id="Создание-таблицы.-1.2.4"><span class="toc-item-num">1.2.4&nbsp;&nbsp;</span>Создание таблицы.</a></span></li><li><span><a href="#Вывод:" data-toc-modified-id="Вывод:-1.2.5"><span class="toc-item-num">1.2.5&nbsp;&nbsp;</span>Вывод:</a></span></li></ul></li></ul></li><li><span><a href="#Ссылка-на-дашборд-на-сайте-Tableau-Public:" data-toc-modified-id="Ссылка-на-дашборд-на-сайте-Tableau-Public:-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Ссылка на дашборд на сайте Tableau Public:</a></span></li><li><span><a href="#Ссылка-на-pdf-файл-с-презентацией:" data-toc-modified-id="Ссылка-на-pdf-файл-с-презентацией:-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Ссылка на pdf-файл с презентацией:</a></span></li></ul></div>

# Текущий уровень потребительской лояльности (Net Promoter Score)

## Описание проекта

Заказчик этого исследования — большая телекоммуникационная компания, которая оказывает услуги на территории всего СНГ. Перед компанией стоит задача определить текущий уровень потребительской лояльности, или NPS (от англ. Net Promoter Score), среди клиентов из России.

Чтобы определить уровень лояльности, клиентам задавали классический вопрос: «Оцените по шкале от 1 до 10 вероятность того, что вы порекомендуете компанию друзьям и знакомым».

Компания провела опрос и попросила вас подготовить дашборд с его итогами. Большую базу данных для такой задачи разворачивать не стали и выгрузили данные в SQLite.

Чтобы оценить результаты опроса, оценки обычно делят на три группы:
- 9-10 баллов — «cторонники» (англ. promoters);
- 7-8 баллов — «нейтралы» (англ. passives);
- 0-6 баллов — «критики» (англ. detractors).
Итоговое значение NPS рассчитывается по формуле: % «сторонников» - % «критиков».

Таким образом, значение этого показателя варьируется от -100% (когда все клиенты «критики») до 100% (когда все клиенты лояльны к сервису). Но это крайние случаи, которые редко встретишь на практике.

Интерпретируя результаты NPS-опросов, следует также помнить, что само значение мало о чём говорит. Однако исследования показывают, что клиенты-сторонники полезны любому бизнесу. Они чаще других повторно совершают покупки, активнее тестируют обновления и приводят в сервис своих друзей и знакомых. Поэтому NPS остаётся одной из важнейших метрик бизнеса.

## Установка библиотек и загрузка данных.

### Установка библиотек.

Для работы с данными будем использовать представленные ниже библиотеки.

In [1]:
# Установка библиотек
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
import plotly.express as px 

### Загрузка данных

В переменной path_to_db содержится путь к файлу на локальном устройстве во время выполнения проекта, вы можете указать свой, заменив значение.

In [2]:
path_to_db = '/users/danielnodelman/yandex_ds/yandex_projects/telecomm_project/telecomm_csi.db' # путь к файлу (можно заменить) #/datasets/telecomm_csi.db
engine = create_engine(f'sqlite:///{path_to_db}', echo = False)

### Формирование SQL запроса.

Формируем запрос на языке SQL. Сохраняем его у переменной query.

In [3]:
# Переменная с запросом
query = """ 
SELECT user_id,
       ABS(lt_day) AS lt_day,
       CASE WHEN lt_day < 365 THEN 'new'
            ELSE 'old'
            END as is_new,
       age,
       CASE WHEN gender_segment = 1 THEN 'man'
            WHEN gender_segment = 0 THEN 'woman'
            ELSE 'n/a'
            END as gender_segment,
       LOWER(os_name) AS os_name,
       LOWER(cpe_type_name) AS cpe_type_name,
       country,
       city,
       SUBSTR(age_segment.title,3) as age_segment,
       SUBSTR(traffic_segment.title,3) as traffic_segment,
       SUBSTR(lifetime_segment.title,3) as life_time_segment,
       nps_score,
       CASE WHEN nps_score > 8 THEN 'promoters'
            WHEN nps_score > 6 THEN 'passives'
            ELSE 'detractors'
            END as nps_group
FROM user
INNER JOIN location USING(location_id)
INNER JOIN age_segment USING(age_gr_id)
INNER JOIN traffic_segment USING(tr_gr_id)
INNER JOIN lifetime_segment USING(lt_gr_id)
""" 

### Создание таблицы.

Таблица будет сохранена в переменной df. Изменим формат файла на csv. Выводим первые 15 строк.

In [4]:
# Прочитываем таблицу и сохраняем её в переменной.
df = pd.read_sql(query, engine)
# Записываем как csv
df.to_csv('telecomm_csi_tableau.csv', index=False)
# Выводим первые 15 строк
df.head(15)

,user_id,lt_day,is_new,age,gender_segment,os_name,cpe_type_name,country,city,age_segment,traffic_segment,life_time_segment,nps_score,nps_group
0,A001A2,2320,old,45.0,man,android,smartphone,Россия,Уфа,45-54,1-5,36+,10,promoters
1,A001WF,2344,old,53.0,woman,android,smartphone,Россия,Киров,45-54,1-5,36+,10,promoters
2,A003Q7,467,old,57.0,woman,android,smartphone,Россия,Москва,55-64,20-25,13-24,10,promoters
3,A004TB,4190,old,44.0,man,ios,smartphone,Россия,РостовнаДону,35-44,0.1-1,36+,10,promoters
4,A004XT,1163,old,24.0,woman,android,smartphone,Россия,Рязань,16-24,5-10,36+,10,promoters
5,A005O0,5501,old,42.0,man,android,smartphone,Россия,Омск,35-44,5-10,36+,6,detractors
6,A0061R,1236,old,45.0,woman,android,smartphone,Россия,Уфа,45-54,10-15,36+,10,promoters
7,A009KS,313,new,35.0,woman,android,smartphone,Россия,Москва,35-44,45-50,7-12,10,promoters
8,A00AES,3238,old,36.0,man,android,smartphone,Россия,СанктПетербург,35-44,1-5,36+,10,promoters
9,A00F70,4479,old,54.0,man,android,smartphone,Россия,Волгоград,45-54,15-20,36+,9,promoters


### Вывод:

 - Данные выгружены 
 - Таблицы объединены
 - Формат таблицы соответсвует ТЗ

- Отрицательные значения заменены на модуль
- Занчения в столбцах преведены к единому стилю.

# Ссылка на дашборд на сайте Tableau Public:

https://public.tableau.com/views/TelecommunicationcompanyNetPromoterScore/sheet7?:language=en-US&publish=yes&:display_count=n&:origin=viz_share_link

# Ссылка на pdf-файл с презентацией:

https://drive.google.com/file/d/1dajhES0K3Su-UOaA-dhaiBnQ4NiEKQG1/view?usp=sharing